## Data Cleaning & Modelling
In this notebook we seek to aggregate the data we've scraped and gathered, clean it, and prepare it for use in our modelling (whether primarily statistical or ML-based). Data has been extracted from multiple sources (primarily wikipedia tables), saved in excel flat files. We then build our recommendation system based on selected specifications.

In [ ]:
# import packages
from fuzzywuzzy import process, fuzz
from itertools import repeat
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper, gen_features
import miceforest as mf
import pandas as pd
import numpy as np
import gower
import re
import os

# are we loading up from saved data
load_data = False

# set params for imputation
seed = 100
iters = 30
datasets = 10
kernel_path = "../Data/mice_kernel"

# set path for saving/loading aggregate data
aggregate_path = "../Data/Aggregate_Destination_Data.xlsx"

In [ ]:
# load previously saved kernel and data
if load_data:
    aggregate_data = pd.read_excel(aggregate_path, index_col=0)
    kernel = mf.load_kernel(kernel_path)

### Load Raw Data
Here we load raw data from excel files in the designated directory.

In [ ]:
# Retrieve master list of destinations
destinations = pd.read_excel(
    io="../Data/Travel_Destinations.xlsx",
    header=None
)[0].values.tolist()

# Retrieve file names and contruct paths for all raw data files
dir_name = "raw_data"
data_files = os.listdir(dir_name)
data_paths = list()
for file_name in data_files:
    data_paths.append((file_name, os.path.join(dir_name, file_name)))

### Fuzzy matching

The goal here is that for each table in an excel flat file, given an instance in the data which has an associated destination, we want to compare the destination in the data with our master list of destinations and use fuzzy matching to select the most appropriate match. We replace the original destination with the matched destination. Upon completion, each data set will have the same set of destinations listed, making aggregation more accurate and feasible.

Fuzzy matching typically uses the Levenshtein distance (aka the edit distance) or Indel distance as a basis. Both compute the number of operations, or edits, required to transform one string into another. The deletion, insertion, or substitution of a character all result in +1 to the Leneshtein distance (all actions are weighted equally). Related, the Indel distance is similar to the Levenshtein distance except it does not permit substitutions. Effectively, any substitution that would have incurred a cost of +1 is replaced by an insertion and deletion with cost +2, or equivalently, a substitution is given a weight of +2. The normalized distance for Levenshtein is computed as $$\frac{distance}{max(len(string1),\ len(string2)}$$ whereas the normalized distance for Indel is computed as $$\frac{distance}{len(string1)\ +\ len(string2)}$$

The similarity between two strings is $1-distance$. When using a specific ratio in the FuzzyWuzzy package, the function preprocesses the input strings in some way and then computes the scaled Levenshtein similarity. The Token Sort Ratio splits a string into its tokens/words, makes them lowercase and removes punctuations, then sorts the tokens alphabetically and joins them prior to computing the similarity.

In [ ]:
fuzzy_dataframes = list()
fuzzy_path = "fuzzy_data"
if not os.path.exists(fuzzy_path):
    os.mkdir(fuzzy_path)

# for every file and table within
table_count = 0
for data_path in data_paths:
    file_name = data_path[0]
    file_path = data_path[1]
    table_count += 1
    
    # open the file
    with open(file_path, "rb") as f:
        # read the file and extract its list of destinations
        unmatched_data = pd.read_excel(f, index_col=0)
        unmatched_destinations = unmatched_data.index.values.tolist()
        
        # perform fuzzy match 
        matched_destinations = list(map(
            process.extractOne,  # function to be applied
            unmatched_destinations,  # applied to 
            repeat(destinations),  # compared against
            repeat(process.default_processor),  # use default processor each time
            repeat(fuzz.token_sort_ratio),  # use token sort ratio instead of default
            repeat(75)  # similarity must be higher than 50% to be considered a match
        ))

        # debugging the fuzzy match logic
        unmatched_count = matched_destinations.count(None)
        print(f"Table {table_count} did not match {unmatched_count} entries")
        if unmatched_count > 0:
            unmatched_indices = [i for i, match in enumerate(matched_destinations) if match == None]
        
        # check if any entries are None indicating no match, if so, set to previous value 
        matched_destinations = [unmatched_destinations[index] if match == None else match[0] for index, match in enumerate(matched_destinations)]

        # debugging the fuzzy match logic
        if unmatched_count > 0:
            for i in range(unmatched_count):
                print(f"---{matched_destinations[unmatched_indices[i]]}")
                destinations.append(matched_destinations[unmatched_indices[i]])  # add legitimate unmatched destinations to main list
        
        # reindex the data to the fuzzy matched destinations
        matched_data = unmatched_data.set_index(pd.Index(matched_destinations))

        # save the file name in the fuzzy folder
        matched_data.to_excel(os.path.join(fuzzy_path, file_name))
        fuzzy_dataframes.append(matched_data)

In [ ]:
# outer join all data based on the destinations
aggregate_data = fuzzy_dataframes[2].join(
    fuzzy_dataframes[0:2]+fuzzy_dataframes[3:],
    how="outer",
    sort=True
)

# remove duplicate rows that end up in the data
aggregate_data = aggregate_data[~aggregate_data.index.duplicated(keep='first')]

# drop Vatican City
aggregate_data.drop(labels="Vatican City", axis=0, inplace=True)

# convert object columns to categories
for col in aggregate_data.dtypes[aggregate_data.dtypes == aggregate_data.dtypes[0]].index:
    aggregate_data[col] = aggregate_data[col].astype("category")

# get rid of all weird characters from column names 
aggregate_data = aggregate_data.rename(columns = lambda x: x.replace('–', '-'))
aggregate_data = aggregate_data.rename(columns = lambda x: re.sub('[^A-Za-z0-9_%()$./<-]+', '', x))

# save aggregate data
aggregate_data.to_excel(aggregate_path, index=True)

### Imputation

Here we perform imputation to fill in missing values. There are no destinations in this data that have all missing values. We do not drop any destinations from the analysis as we prefer to have a broader set to recommend to end-users.

In this analysis we use multiple imputation by chained equations (MICE) as our imputation method, due to its attempts to account for the uncertainty in the imputed value by provingd both within-imputation and between-imputation variability, a characteristic which makes multiple imputation preferable over single imputation in most circumstances.

There continues to be discussion regarding the order of operations between certain data preprocessing steps, namely scaling data via standardization/normalization and imputation. Which of the two procedures comes first may largely depend on the intent of the analysis, imputation method, and modelling that is done downstream. MICE is not necessarily sensitive to scale and with the use of predictive mean matching we are able to avoid imputing nonsensical values (e.g., negative population for a destination).

In [ ]:
# create kernel for multiple imputation
kernel = mf.ImputationKernel(
    data = aggregate_data,
    datasets = datasets,
    train_nonmissing=False,
    mean_match_scheme=mf.mean_match_shap,
    save_all_iterations=True,
    save_models=1,
    copy_data=True,
    random_state=seed
)

# iteratively impute data and save final kernel
kernel.mice(iters)
kernel.save_kernel(kernel_path)

### Recommendation System

Recommender systems come in various shapes. 

Some necessitate that you have information about users and their feelings towards the items being recommended, typically collected in the form of user provided ratings. This is typical of approaches based on `collaborative filtering`. From there, memory-based or model-based approaches are specifically taken, but in general, the focus of these methods are in recommending items to users based on the user's similarity to other users, inferring that if users are similar they are likely to like items that other similar users liked. 

There are also `content based systems` which usually do not have as much data on ratings but supplement this with data related to the attributes of the items being recommended. These systems generally try to recommend items similar to those a user has liked before based on their ratings and the attributes of the item. Hybrid systems also exist, which are likely much more robust in practice.

Here, given the raw data and the lack of user ratings on destinations we suffer from a sort of cold-start problem - how do we recommend new destinations to a user if we have no information on how users feel about the destinations we could possibly recommend? As a result, we take a more simplistic approach similar to content based systems. We use distance and similarity measures to determine what to recommend to a user based on their own past travel experiences. Specifically we use Gower's distance which can handle both categorical and continuous data.

In [ ]:
# define travel recommendation function
def get_recommendations(kernel, data_sets, user_destinations, similar="Y", recs=5, ):
    """
    A function which generates recommendations for travel destinations based on user input
    using Gower's distance and voting ensemble principles for aggregating results across
    imputed data sets.


    Parameters
    ----------
    kernel (miceforest.ImputationKernel) : kernel dataset created by the miceforest package 
    data_sets (int) : count of data sets created through MICE and stored in kernel
    user_destinations (np.ndarray) : array of user provided destinations
    similar (str) : 'Y' or 'N' indicating user wants similar or dissimilar recommendations
    recs (int) : number of recommendations wanted by the user
    """
    
    recommendations = np.array(list())
    numeric_cols = [[col] for col in kernel.working_data.select_dtypes(include=[np.number]).columns.values]

    for data in range(data_sets):
        imputed_df = kernel.complete_data(data)

        # standardize features using feature map of numerical columns to 
        # ensure distance measure is not influenced by scale of features
        feature_map = gen_features(columns=numeric_cols, classes=[StandardScaler])
        mapper = DataFrameMapper(
            feature_map,
            default=None,
            input_df=True,
            df_out=True
        )
        imputed_df = mapper.fit_transform(imputed_df)
        
        # get the df indices of the destinations provided by the user
        user_dest_idx = np.array([imputed_df.index.get_loc(loc) for loc in user_destinations])
        
        # get the computed distances for the provided destinations
        dist = gower.gower_matrix(np.asarray(imputed_df))[user_dest_idx, :]
        
        # get the indices of the user provided destinations which should not be recommended again
        excl_dest = (dist == 0).sum(axis=0).nonzero()[0]
    
        
        if similar == "Y":
            # retrieve the top n most similar destinations - minimize the sum of distances across the provided locations
            temp_recs = np.argpartition(dist.sum(axis=0), (recs+3))[:(recs+3)]
        else:
            # retrieve the top n least similar destinations - maximize the sum of distances across the provided locations
            temp_recs = np.argpartition(dist.sum(axis=0), -(recs+3))[-(recs+3):]
        
        # exclude any of the user provided destinations
        for loc in excl_dest:
            temp_recs = temp_recs[temp_recs != loc]
        
        # provide the final list of recommendations that will be output to the user
        final_recs = temp_recs[:recs]
        final_recs = np.array(imputed_df.index[final_recs])
    
        # aggregate recos across data sets
        recommendations = np.append(recommendations, final_recs)
    
    # select the most frequently recommended locations across data sets
    recommendations = np.unique(recommendations, return_counts=True)
    
    # return list of top n recommendations based on countries provided and similarity/dissimilarity 
    return recommendations


def print_recommendations(recommendations, similar="Y", recs=5):
    """
    A function which outputs the results provided by recommender system.
    
    Parameters
    ----------
    recommendations (np.ndarray) : array of recommended destinations
    similar (str) : 'Y' or 'N' indicating user wants similar or dissimilar recommendations
    recs (int) : number of recommendations wanted by the user
    """
    recommendations = recommendations[0][np.argsort(recommendations[1])[-recs:]]

    if similar == "Y":
        similar = "similarity"
    else:
        similar = "dissimilarity"
    
    print(f"Thanks for your patience, here are {recs} recommendations based on {similar}:")
    for i, loc in enumerate(recommendations):
        print(f"{i+1}. {loc}")

In [ ]:
# example user input
user_destinations = np.array(["Cuba", "Mexico"])  # user provided destinations
num_destinations = len(user_destinations)
similar = "Y"  # user desires similar recommendations
recs = 5  # number of recommendations desired by the user
kernel = mf.load_kernel("../Data/mice_kernel")

In [ ]:
recommendations = get_recommendations(
    kernel,
    datasets,
    user_destinations,
    similar, 
    recs
) 

print_recommendations(recommendations, similar, recs)

### Visualization

Using the plotly library, we visualize the results of the recommendation system on an interactive map that renders in browser

In [ ]:
import pycountry
import plotly.express as px
import plotly.io as pio

# pio.renderers.default = "browser" to make viz render in browser

countries = kernel.working_data.index.values
unmapped = {
    "Bosnia": "Bosnia and Herzegovina",
    "Brunei": "Brunei Darussalam",
    "Cape Verde": "Cabo Verde",
    "Caribbean Netherlands": "Netherlands",
    "Côte d’Ivoire": "Côte d'Ivoire",
    "Democratic Republic of Congo": "Congo, The Democratic Republic of the",
    "East Timor": "Timor-Leste",
    "Falkland Islands": "Falkland Islands (Malvinas)",
    "U.S. Virgin Islands": "Virgin Islands, U.S.",
    "Syria": "Syrian Arab Republic",
    "São Tomé and Príncipe": "Sao Tome and Principe",
    "Sint Maarten": "Sint Maarten (Dutch part)",
    "Saint Martin": "Saint Martin (French part)",
    "Saint Helena": "Saint Helena, Ascension and Tristan da Cunha",
    "Russia": "Russian Federation",
    "Micronesia": "Micronesia, Federated States of",
    "Palestine": "Palestine, State of",
    "Laos": "Lao People's Democratic Republic",
    "Kosovo": "Serbia",  
    "Iran": "Iran, Islamic Republic of" 
}

codes = [pycountry.countries.lookup(unmapped[country]).alpha_3 if country in unmapped else pycountry.countries.lookup(country).alpha_3 for country in recommendations[0]]
normalized_freq = (recommendations[1] - min(recommendations[1])) / (max(recommendations[1]) - min(recommendations[1]))
reco_freq = pd.DataFrame(
    data = {
        "Country": recommendations[0],
        "ISO_3": codes, 
        "Recommendation intensity": normalized_freq
    }
).iloc[np.argsort(recommendations[1])[-recs:]][:]

In [ ]:
title_str = "You have visited, or have an interest in, "
if len(user_destinations) == 1:
    title_str += f"{user_destinations[0]}"
elif len(user_destinations) == 2:
    title_str += f"{user_destinations[0]} and {user_destinations[1]}"
else:
    title_str += f"{user_destinations[0]}, {user_destinations[1]}, and {user_destinations[2]}"

fig = px.choropleth(
    reco_freq, 
    locations='ISO_3', 
    color='Recommendation intensity', 
    range_color=[0,1],
    hover_name='Country',
    hover_data={
        "ISO_3": False,
        "Recommendation intensity": ":.2f"
    },
    projection='equirectangular', 
    title="Relative strength in the recommendation of each travel destination<br><sup>"+title_str,
    color_continuous_scale="viridis"
)
fig.update_layout(
    title=dict(font=dict(size=26), x=0.015,y=.9625),
    margin=dict(l=15,r=10,b=30,t=70),
    coloraxis_colorbar=dict(
    title="Recommendation<br>intensity",
    thicknessmode="pixels",
    lenmode="pixels",
    yanchor="top",
    y=1,
    xanchor="left",
    x=.01,
    tickvals=[0,0.5,1],
    ticktext=["Low", "Medium", "High"],
))
fig.show()